<a href="https://colab.research.google.com/github/hewness/FL2024.B69.DAT.8551.21_Final_Project/blob/main/FL2024_B69_DAT_8551_21_Part_B_Data_Validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FL2024.B69.DAT.8551.21 - Part B - Data Validation

The purpose of this notebook is validate the 311 service request data that was extracted from NYC OpenData.

In [ ]:
import os
import pandas as pd
import numpy as np
import plotly.express as px
from datetime import datetime
import calendar

from google.colab import drive, data_table

data_table.enable_dataframe_formatter()

## Mount Colab environment to Google Drive

The extracted data was uploaded to Google Drive to easily import it into the Colab environment. The below mounts the drive and imports the data into a pandas dataframe.

In [ ]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
BASE_DATA_DIR = '/content/gdrive/MyDrive/FL2024.B69.DAT.8551.21/final_project/data'
SERVICE_REQUESTS_FILENAME = '311_Service_Requests_2021-2023.csv'

service_requests_path = os.path.join(BASE_DATA_DIR, SERVICE_REQUESTS_FILENAME)

In [ ]:
import dask.dataframe as dd

service_requests_dd = dd.read_csv(service_requests_path, dtype={
    "Unique Key": object,
    "Incident Zip": object,
    "BBL": object,
    "Bridge Highway Direction": object,
    "Bridge Highway Name": object,
    "Bridge Highway Segment": object,
    "Due Date": object,
    "Facility Type": object,
    "Road Ramp": object,
    "Taxi Company Borough": object,
    "Taxi Pick Up Location": object,
    "Vehicle Type": object,
    "X Coordinate (State Plane)": object,
    "Y Coordinate (State Plane)": object,
    "Latitude": object,
    "Longitude": object
})

## Basic EDA Checks

In [ ]:
data_table.DataTable(service_requests_dd.head(), max_columns=41)

,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,...,Vehicle Type,Taxi Company Borough,Taxi Pick Up Location,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Latitude,Longitude,Location
0,59889383,12/31/2023 11:59:42 PM,01/01/2024 01:51:01 AM,NYPD,New York City Police Department,Noise - Street/Sidewalk,Loud Music/Party,Street/Sidewalk,11375,63-10 108 STREET,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,40.734694673156454,-73.85052125577377,"(40.734694673156454, -73.85052125577377)"
1,59887573,12/31/2023 11:59:39 PM,01/19/2024 02:37:37 PM,EDC,Economic Development Corporation,Noise - Helicopter,Other,Above Address,10023,25 WEST 73 STREET,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,40.77720102455921,-73.976158989108,"(40.77720102455921, -73.976158989108)"
2,59892892,12/31/2023 11:59:29 PM,01/01/2024 04:06:24 AM,NYPD,New York City Police Department,Blocked Driveway,Partial Access,Street/Sidewalk,11355,143-34 BARCLAY AVENUE,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,40.759286493466355,-73.82261386696051,"(40.759286493466355, -73.82261386696051)"
3,59893860,12/31/2023 11:59:29 PM,01/01/2024 01:51:32 AM,NYPD,New York City Police Department,Noise - Street/Sidewalk,Loud Music/Party,Street/Sidewalk,11374,65-09 99 STREET,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,40.72937885745978,-73.85543290785074,"(40.72937885745978, -73.85543290785074)"
4,59887231,12/31/2023 11:59:23 PM,01/01/2024 12:13:30 AM,NYPD,New York City Police Department,Noise - Street/Sidewalk,Loud Music/Party,Street/Sidewalk,11232,870 42 STREET,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,40.64472479285036,-73.9976217135385,"(40.64472479285036, -73.9976217135385)"


## Check for Mixed Case Categorical Values

We find whether there are mixed cased categorical value for categorical columns in the 311 Service Data Request that we are interested in. This can cause problems with grouping values in our visualizations depending on whether we use them.

The commands are separated out as they are long running and we want to be able to rerun them separately in case an error occurs (e.g. Colab crashes, etc.).

In [ ]:
def check_categorical_data(categorical_series):
    category_set = set()

    category_case_diff = []

    for category_value in categorical_series:
        if pd.isnull(category_value):
            continue
        else:
            category_value_lower = category_value.lower()

            if category_value_lower not in category_set:
                category_set.add(category_value_lower)
            else:
                category_case_diff.append(category_value_lower)

    print('===CHECKING MIXED CASE VALUES===')

    for category_case_diff_value in category_case_diff:
        matching_categories = []
        for category_value in categorical_series:
            if not pd.isnull(category_value) and category_case_diff_value == category_value.lower():
                matching_categories.append(category_value)

        print(matching_categories)

    print('===CHECKING SUPPORTED NULL VALUES===')

    supported_null_values = []

    for category_value in categorical_series:
        if pd.isnull(category_value):
            supported_null_values.append(None)
        else:
            category_value_lower = category_value.lower()
            if category_value in ['n/a', 'na']:
                supported_null_values.append(category_value_lower)

    if supported_null_values:
        print(supported_null_values)

    print('===CHECKING FOR SIMILAR VALUES===')

    for category_value in categorical_series:
        if pd.isnull(category_value):
            continue

        category_value_lower = category_value.lower()

        for category_value2 in categorical_series:
            if pd.isnull(category_value2):
                continue
            else:
                category_value2_lower = category_value2.lower()

                if len(category_value2) > len(category_value_lower) and category_value2_lower.startswith(category_value_lower):
                    print(f'{category_value} is similar to {category_value2}')


In [ ]:
agency_categorical_values = service_requests_dd['Agency'].unique().compute()

In [ ]:
check_categorical_data(agency_categorical_values)

===CHECKING MIXED CASE VALUES===
===CHECKING SUPPORTED NULL VALUES===
===CHECKING FOR SIMILAR VALUES===


In [ ]:
agency_name_categorical_values = service_requests_dd['Agency Name'].unique().compute()

In [ ]:
check_categorical_data(agency_name_categorical_values)

===CHECKING MIXED CASE VALUES===
===CHECKING SUPPORTED NULL VALUES===
===CHECKING FOR SIMILAR VALUES===


In [ ]:
complaint_type_categorical_values = service_requests_dd['Complaint Type'].unique().compute()

In [ ]:
check_categorical_data(complaint_type_categorical_values)

===CHECKING MIXED CASE VALUES===
['PLUMBING', 'Plumbing']
['Elevator', 'ELEVATOR']
['PAINT/PLASTER', 'Paint/Plaster']
['DOOR/WINDOW', 'Door/Window']
['FLOORING/STAIRS', 'Flooring/Stairs']
['GENERAL', 'General']
['WATER LEAK', 'Water Leak']
['UNSANITARY CONDITION', 'Unsanitary Condition']
['APPLIANCE', 'Appliance']
['HEAT/HOT WATER', 'Heat/Hot Water']
['ELECTRIC', 'Electric']
['SAFETY', 'Safety']
['OUTSIDE BUILDING', 'Outside Building']
['Mold', 'MOLD']
===CHECKING SUPPORTED NULL VALUES===
===CHECKING FOR SIMILAR VALUES===
ELECTRIC is similar to Electrical
Sewer is similar to Sewer Maintenance
Noise is similar to Noise - Street/Sidewalk
Noise is similar to Noise - Helicopter
Noise is similar to Noise - Residential
Noise is similar to Noise - Commercial
Noise is similar to Noise - Vehicle
Noise is similar to Noise - Park
Noise is similar to Noise - House of Worship
GENERAL is similar to General Construction/Plumbing
Dirty Condition is similar to Dirty Conditions
Missed Collection is simi

In [ ]:
descriptor_series = service_requests_dd['Descriptor'].unique().compute()

In [ ]:
check_categorical_data(descriptor_series)

===CHECKING MIXED CASE VALUES===
['Lighting', 'LIGHTING']
['SEWAGE', 'Sewage']
['DOOR', 'Door']
['CEILING', 'Ceiling']
['WINDOW FRAME', 'Window Frame']
['FLOOR', 'Floor']
['CABINET', 'Cabinet']
['WALL', 'Wall']
['BATHTUB/SHOWER', 'Bathtub/Shower']
['BELL/BUZZER/INTERCOM', 'Bell/Buzzer/Intercom']
['HEAVY FLOW', 'Heavy Flow']
['DOOR FRAME', 'Door Frame']
['RADIATOR', 'Radiator']
['WATER SUPPLY', 'Water Supply']
['MOLD', 'Mold']
['BASIN/SINK', 'Basin/Sink']
['DAMP SPOT', 'Damp Spot']
['REFRIGERATOR', 'Refrigerator']
['WINDOW PANE', 'Window Pane']
['TOILET', 'Toilet']
['SLOW LEAK', 'Slow Leak']
['ENTIRE BUILDING', 'Entire Building']
['APARTMENT ONLY', 'Apartment Only']
['COOKING GAS', 'Cooking Gas']
['POWER OUTAGE', 'Power Outage']
['FIRE ESCAPE', 'Fire Escape']
['ELECTRIC/GAS RANGE', 'Electric/Gas Range']
['WINDOW/FRAME', 'Window/Frame']
['SMOKE DETECTOR', 'Smoke Detector']
['CARBON MONOXIDE DETECTOR', 'Carbon Monoxide Detector']
['WINDOW GUARD BROKEN/MISSING', 'Window Guard Broken/Missin

In [ ]:
check_categorical_data(service_requests_dd['Location Type'].unique().compute())

===CHECKING MIXED CASE VALUES===
['RESIDENTIAL BUILDING', 'Residential Building']
['3+ Family ApT', '3+ Family Apt']
===CHECKING SUPPORTED NULL VALUES===
[None]
===CHECKING FOR SIMILAR VALUES===
RESIDENTIAL BUILDING is similar to Residential Building/House
Street is similar to Street/Sidewalk
Street is similar to Street/Curbside
Street is similar to Street Vendor
Street is similar to Street Fair Vendor
Street is similar to Street Area
Street is similar to Street Address
Business is similar to Business Open to the Public
Residential Building is similar to Residential Building/House
Subway is similar to Subway Station
Residential is similar to Residential Building/House
Residential is similar to RESIDENTIAL BUILDING
Residential is similar to Residential Building
Residential is similar to Residential Property
Vacant Lot is similar to Vacant Lot/Property
Park is similar to Park/Playground
Park is similar to Parking Lot
Park is similar to Parking Lot/Garage
Other is similar to Other (Explai

In [ ]:
check_categorical_data(service_requests_dd['City'].unique().compute())

===CHECKING MIXED CASE VALUES===
['BROOKLYN', 'Brooklyn', 'brooklyn']
['MANHATTAN', 'Manhattan', 'manhattan']
['ELMHURST', 'Elmhurst']
['LONG ISLAND CITY', 'Long Island City']
['BRONX', 'Bronx', 'bronx']
['BROOKLYN', 'Brooklyn', 'brooklyn']
['MANHATTAN', 'Manhattan', 'manhattan']
['QUEENS', 'Queens', 'queens']
['NEW YORK', 'New York', 'new york']
['QUEENS', 'Queens', 'queens']
['Farmingdale', 'farmingdale', 'FARMINGDALE']
['BRONX', 'Bronx', 'bronx']
['PELHAM', 'pelham']
['WOODSIDE', 'Woodside']
['Great Neck', 'Great neck', 'GREAT NECK']
['NEW YORK', 'New York', 'new york']
['Buffalo', 'buffalo']
['Island Park', 'island park']
['NEW HYDE PARK', 'New Hyde Park', 'New Hyde park']
['long island', 'Long Island', 'LONG ISLAND']
['Denville', 'denville']
['STATEN ISLAND', 'staten island', 'Staten Island']
['FLORAL PARK', 'floral park', 'Floral Park']
['Queen', 'queen']
['NEW HYDE PARK', 'New Hyde Park', 'New Hyde park']
['Austin', 'austin', 'AUSTIN']
['FLUSHING', 'Flushing']
['ASTORIA', 'Astor

In [ ]:
check_categorical_data(service_requests_dd['Facility Type'].unique().compute())

===CHECKING MIXED CASE VALUES===
===CHECKING SUPPORTED NULL VALUES===
[None]
===CHECKING FOR SIMILAR VALUES===


In [ ]:
check_categorical_data(service_requests_dd['Status'].unique().compute())

===CHECKING MIXED CASE VALUES===
===CHECKING SUPPORTED NULL VALUES===
===CHECKING FOR SIMILAR VALUES===


In [ ]:
check_categorical_data(service_requests_dd['Community Board'].unique().compute())

===CHECKING MIXED CASE VALUES===
===CHECKING SUPPORTED NULL VALUES===
[None]
===CHECKING FOR SIMILAR VALUES===


In [ ]:
check_categorical_data(service_requests_dd['Borough'].unique().compute())

===CHECKING MIXED CASE VALUES===
===CHECKING SUPPORTED NULL VALUES===
[None]
===CHECKING FOR SIMILAR VALUES===


In [ ]:
check_categorical_data(service_requests_dd['Open Data Channel Type'].unique().compute())

===CHECKING MIXED CASE VALUES===
===CHECKING SUPPORTED NULL VALUES===
===CHECKING FOR SIMILAR VALUES===


In [ ]:
check_categorical_data(service_requests_dd['Park Facility Name'].unique().compute())

===CHECKING MIXED CASE VALUES===
['Rosalyn Yalow Charter School', 'ROSALYN YALOW CHARTER SCHOOL']
['Hellenic Classical Charter School', 'HELLENIC CLASSICAL CHARTER SCHOOL']
['Kings Collegiate Charter School', 'KINGS COLLEGIATE CHARTER SCHOOL']
['Canarsie Ascend Charter School', 'CANARSIE ASCEND CHARTER SCHOOL']
['Harlem Prep Charter School', 'HARLEM PREP CHARTER SCHOOL']
['Challenge Preparatory Charter School', 'CHALLENGE PREPARATORY CHARTER SCHOOL']
['Global Community Charter School', 'GLOBAL COMMUNITY CHARTER SCHOOL']
['PAVE Academy Charter School', 'PAVE ACADEMY CHARTER SCHOOL']
['Bronx Charter School for the Arts', 'BRONX CHARTER SCHOOL FOR THE ARTS']
['VOICE Charter School of New York', 'VOICE CHARTER SCHOOL OF NEW YORK']
['South Bronx Classical Charter School', 'SOUTH BRONX CLASSICAL CHARTER SCHOOL']
['Brilla Veritas Charter School', 'BRILLA VERITAS CHARTER SCHOOL']
===CHECKING SUPPORTED NULL VALUES===
[None]
===CHECKING FOR SIMILAR VALUES===
Central Park is similar to CENTRAL PA

In [ ]:
check_categorical_data(service_requests_dd['Park Borough'].unique().compute())

===CHECKING MIXED CASE VALUES===
===CHECKING SUPPORTED NULL VALUES===
[None]
===CHECKING FOR SIMILAR VALUES===


In [ ]:
check_categorical_data(service_requests_dd['Vehicle Type'].unique().compute())

===CHECKING MIXED CASE VALUES===
===CHECKING SUPPORTED NULL VALUES===
[None]
===CHECKING FOR SIMILAR VALUES===
Car is similar to Car Service
